In [1]:
from xml.dom.minidom import parse, parseString
import pandas as pd
from math import cos, asin, sqrt

gpx = open("walk1.gpx")
dom = parse(gpx)
trkpts = dom.getElementsByTagName("trkpt")

# adapted from stackoverflow https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206
def distanceEachPt(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) #2*R*asin...

def distance(data):
    df = data
    df['lat1']= pd.to_numeric(data['lat'])
    df['lon1']= pd.to_numeric(data['lon'])
    df['lat2']= df['lat1'].shift(-1)
    df['lon2']= df['lon1'].shift(-1)
    df['lat2'].iloc[-1]= df['lat2'].iloc[-2]
    df['lon2'].iloc[-1]= df['lon2'].iloc[-2]
    df['distance']=df.apply(lambda x: distanceEachPt(x['lat1'],x['lon1'],x['lat2'],x['lon2']), axis=1)
    return df['distance'].sum() * 1000
    

In [2]:
lat = []
lon = []
for i in range(len(trkpts)):
    lat.append(trkpts[i].getAttribute("lat"))
    lon.append(trkpts[i].getAttribute("lon"))

walk1 = pd.DataFrame(lat, lon).reset_index()
walk1.columns = ['lat', 'lon']

In [4]:
print('Unfiltered distance: %0.2f' % (distance(walk1)))

Unfiltered distance: 3648.40


/Users/kai/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
from pykalman import KalmanFilter

ModuleNotFoundError: No module named 'pykalman'

In [3]:
import sys
sys.version

'3.6.0 |Anaconda custom (x86_64)| (default, Dec 23 2016, 13:19:00) \n[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]'